In [3]:
from Bio import SeqIO
import pandas as pd

# Add src directory from src
import sys
sys.path.append('../src/Core')

# Import get_metadata file
from get_metadata import get_sequence_metadata

In [4]:
df = get_sequence_metadata("../data/FASTA_samples_no_identical_seqs.fa", save = False)

FileNotFoundError: [Errno 2] No such file or directory: '../data/FASTA_samples_no_identical_seqs.fa'

In [29]:
df.head(10)

,accession,strain,year,month,day,segname,segnum,length
0,X17221,A/CHR/157/83,1983,,,4 (HA),{segment number},1752
1,X05907,A/England/321/1977,1977,,,4 (HA),{segment number},1762
2,V01085,A/Aichi/2/1968,1968,,,4 (HA),{segment number},1765
3,V01086,A/Victoria/3/1975,1975,,,4 (HA),{segment number},1768
4,V01098,A/Victoria/3/1975,1975,,,4 (HA),{segment number},1768
5,V01103,A/NT/60/68/29c,1968,,,4 (HA),{segment number},1765
6,X15281,A/Chile/1/1983,1983,,,6 (NA),{segment number},1461
7,V01088,A/Puerto Rico/8/1934,1934,,,4 (HA),{segment number},1778
8,D13580,A/Izumi/5/65(R),1965,,,4 (HA),{segment number},1773
9,D13579,A/Izumi/5/65,1965,,,4 (HA),{segment number},1773


In [1]:
# Query function
def query_metadata(df, start_year, end_year=2020, strain = "None", seq_name = "None"):
    """
    Helper function to filter metadata dataframe
    :param df: pandas dataframe
           start_year: int
           end_year: (optional) int
           strain: (optional, case sensitive) string 
           seq_name: (optional, case sensitive) string
    """
    # Filter by seqname
    if seq_name != "None":
        df = df.loc[df['seqname'].str.contains(seq_name)]
    # Filter by strain
    if seq_name != "None":
        df = df.loc[df['strain'].str.contains(strain)]
    # Filter by year
    years = [str(x) for x in np.arange(start_year,end_year+1,1)]
    out = df.loc[df['year'] .isin(years)]
    
    return out

query_metadata(df, 2018,strain="Wash", seq_name ="HA").head(10)

NameError: name 'head' is not defined